<a href="https://colab.research.google.com/github/HeatherDriver/IU-Model-Engineering/blob/main/03_Transformations_01_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pickle
import os
from sklearn.decomposition import PCA

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/01_Data/03_Modelling'

data_path = '/content/drive/MyDrive/01_Data/02_Processed/'
pkl_path = '/content/drive/MyDrive/01_Data/03_Modelling/'
images_folder = '/content/drive/MyDrive/02_Docs'

Mounted at /content/drive
/content/drive/MyDrive/01_Data/03_Modelling


In [4]:
# Display all columns, rows and import the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

warnings.filterwarnings('ignore')

In [5]:
# Importing pickle files

def pkl_reader(filename):
  with open(filename, 'rb') as file:
    loaded_object = pickle.load(file)
  return loaded_object

In [6]:
X_for_pca = pkl_reader('X_for_pca.pkl')

In [7]:
# Create test principal components to reduce dimensionality.
pca_variances = []
for _df in X_for_pca:
  X = _df.copy(deep=True)
  X.drop(columns=['day_of_origin', 'ac_registration'], inplace=True)
  pca = PCA(min(X.shape) - 1)
  X_pca = pca.fit_transform(X)
  _ = pd.DataFrame(pca.components_, columns=X.columns)
  var_df = pd.DataFrame(pca.explained_variance_ratio_, columns=['Explained_Variance'])
  var_df['Running_Total'] = var_df['Explained_Variance'].cumsum(axis=0)
  var_df = var_df.reset_index(drop=False)
  var_df['index'] = var_df['index'] + 1
  princ_comp_df = pd.concat([var_df,_], axis=1)
  pca_variances.append(princ_comp_df)

In [8]:
pca_variances[0].head(3)

,index,Explained_Variance,Running_Total,Ac_Type_Code_DH4,Ac_Type_Code_E95,Crew_Group_imputed_no change_0,Crew_Group_imputed_no change_1,Crew_Group_imputed_no change_10,Crew_Group_imputed_no change_2,Crew_Group_imputed_no change_3,Crew_Group_imputed_no change_4,Crew_Group_imputed_no change_5,Crew_Group_imputed_no change_6,Crew_Group_imputed_no change_7,Crew_Group_imputed_no change_8,Crew_Group_imputed_no change_9,ac_registration_quant_transf_actual_duration_0,ac_registration_quant_transf_actual_duration_1,ac_registration_quant_transf_actual_duration_10,ac_registration_quant_transf_actual_duration_2,ac_registration_quant_transf_actual_duration_3,ac_registration_quant_transf_actual_duration_4,ac_registration_quant_transf_actual_duration_5,ac_registration_quant_transf_actual_duration_6,ac_registration_quant_transf_actual_duration_7,ac_registration_quant_transf_actual_duration_8,ac_registration_quant_transf_actual_duration_9,airport_0_AAY,airport_0_CAE,airport_0_COD,airport_0_DAN,airport_0_DAW,airport_0_DEN,airport_0_DUN,airport_0_EAA,airport_0_EAC,airport_0_EAH,airport_0_EAL,airport_0_EAM,airport_0_EME,airport_0_FIN,airport_0_GID,airport_0_HAE,airport_0_JON,airport_0_KET,airport_0_LAL,airport_0_LAR,airport_0_LDA,airport_0_LIH,airport_0_LKE,airport_0_MAD,airport_0_NAE,airport_0_NAN,airport_0_NEJ,airport_0_NJE,airport_0_NOC,airport_0_NOG,airport_0_NOJ,airport_0_POA,airport_0_POB,airport_0_POC,airport_0_POJ,airport_0_POM,airport_0_POR,airport_0_RYD,airport_0_SON,airport_0_STD,airport_0_SUH,airport_0_WEB,airport_0_WED,airport_0_WEJ,airport_0_YOD,airport_0_YOH,airport_10_EAA,airport_10_EAC,airport_10_LDA,airport_10_NOC,airport_10_POC,airport_10_POR,airport_11_POC,airport_1_CAE,airport_1_CHE,airport_1_COD,airport_1_DAN,airport_1_DAW,airport_1_DEN,airport_1_EAA,airport_1_EAC,airport_1_EAE,airport_1_EAH,airport_1_EAL,airport_1_EAM,airport_1_EAS,airport_1_EAT,airport_1_ELE,airport_1_ELI,airport_1_EME,airport_1_FIN,airport_1_GAD,airport_1_GID,airport_1_HAE,airport_1_JAW,airport_1_JOG,airport_1_JON,airport_1_KEG,airport_1_KET,airport_1_LAD,airport_1_LAG,airport_1_LAK,airport_1_LAL,airport_1_LAR,airport_1_LAT,airport_1_LDA,airport_1_LIH,airport_1_LKE,airport_1_LLN,airport_1_MAD,airport_1_MET,airport_1_MIH,airport_1_MIT,airport_1_NAN,airport_1_NEA,airport_1_NEH,airport_1_NEJ,airport_1_NES,airport_1_NEW,airport_1_NOA,airport_1_NOD,airport_1_NOJ,airport_1_NOR,airport_1_NRI,airport_1_PAY,airport_1_POA,airport_1_POC,airport_1_POJ,airport_1_POR,airport_1_PRO,airport_1_RAN,airport_1_REH,airport_1_RIH,airport_1_ROT,airport_1_RUR,airport_1_RYD,airport_1_SOC,airport_1_SOL,airport_1_SON,airport_1_SOV,airport_1_STE,airport_1_SUH,airport_1_VAG,airport_1_WEA,airport_1_WEB,airport_1_WED,airport_1_WEJ,airport_1_WES,airport_1_WEY,airport_1_YOD,airport_1_YOH,airport_2_AAY,airport_2_CAE,airport_2_CHE,airport_2_DAN,airport_2_DAW,airport_2_DEN,airport_2_DUN,airport_2_EAA,airport_2_EAC,airport_2_EAE,airport_2_EAH,airport_2_EAL,airport_2_EAM,airport_2_EAS,airport_2_EAT,airport_2_ERN,airport_2_FIN,airport_2_GID,airport_2_HAE,airport_2_JOG,airport_2_JON,airport_2_JUH,airport_2_KAE,airport_2_KEG,airport_2_KET,airport_2_LAD,airport_2_LAG,airport_2_LAL,airport_2_LAR,airport_2_LAS,airport_2_LDA,airport_2_LIH,airport_2_LKE,airport_2_LLN,airport_2_MAD,airport_2_MAH,airport_2_MAR,airport_2_MET,airport_2_MIH,airport_2_MIT,airport_2_MON,airport_2_NAA,airport_2_NAN,airport_2_NEB,airport_2_NEJ,airport_2_NES,airport_2_NEW,airport_2_NOC,airport_2_NOD,airport_2_NOF,airport_2_NOJ,airport_2_PAY,airport_2_PHN,airport_2_POA,airport_2_POB,airport_2_POC,airport_2_POJ,airport_2_POR,airport_2_PRO,airport_2_RAN,airport_2_RIH,airport_2_RUR,airport_2_RYD,airport_2_SOC,airport_2_SOL,airport_2_SON,airport_2_STD,airport_2_SUH,airport_2_VAG,airport_2_WEA,airport_2_WEB,airport_2_WED,airport_2_WEJ,airport_2_WES,airport_2_WHD,airport_2_YOD,airport_2_YOH,airport_3_AAY,airport_3_CAE,airport_3_COD,airport_3_DAN,airport_3_DAW,airport_3_DEN,airport_3_EAA,airport_3_EAC,airport_3_EAH,airport_3_EAJ,airport

In [9]:
y_names = ['actual_duration', 'dep_delay', 'Act_Groundtime_imputed']

In [10]:
# Can create a dictionary per dependent variable to get to 95% explained variance.

def returns_95_perc_PCA_components():
  my_list = ['actual_duration', 'dep_delay', 'Act_Groundtime_imputed']
  pca_nums = dict()
  for title, pca_df in zip(my_list, pca_variances):
    components_criteria = pca_df.loc[pca_df['Running_Total'] <= 0.951]['index']
    components = components_criteria.tail(1).values[0]
    pca_nums.update({title : components})
  return pca_nums

In [11]:
# Visualise the number of components and cumulative % explained variance

pca_nums = returns_95_perc_PCA_components()

fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('actual_duration', 'dep_delay', 'Act_Groundtime_imputed'),
                    specs=[[{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}]]
                    )

for y_name, (i, df) in zip(y_names, enumerate(pca_variances, start=1)):
  fig.add_trace(
      go.Bar(x=df['index'].to_list(), y=df['Running_Total'].to_list(), name="Explained Variance", showlegend=False),
      row=1, col=i, secondary_y=False
      )

  fig.add_trace(
      go.Scatter(x=df['index'].to_list(), y=df['Running_Total'].to_list(), name="Running Total", showlegend=False),
      row=1, col=i, secondary_y=True
      )
  fig.update_xaxes(title_text="Number PCA components", row=1, col=i)

  # Add vertical line at specific x position
  fig.add_vline(x=pca_nums[y_name], line_width=1.5, line_dash="dot", line_color="red", annotation_text=str(pca_nums[y_name]) + ' components', annotation_position="bottom right",
                row=1, col=i)

  # Add horizontal line at y=0.95
  fig.add_shape(type="line", x0=df['index'].min(), x1=df['index'].max(), y0=0.95, y1=0.95,
                line=dict(color="red", width=1.5, dash="dot"), row=1, col=i)

# Update title
fig.update_layout(
    title_text="Cumulative explained variance by number of PCA components for each predicted variable",
)
fig.update_layout(title=None)
fig.write_image(images_folder + "/PCA_components_vars.png")
fig.show()

In [12]:
comp_var = []
for _df in pca_variances:
  df = _df.copy(deep=True)
  df.drop(columns=['Running_Total'], inplace=True)
  _df = pd.melt(df, id_vars=['index', 'Explained_Variance'], value_vars=df.columns[3:])
  _df = _df.sort_values(by=['index', 'Explained_Variance', 'value'], ascending=[True, False, False])
  _df = _df.loc[_df['index'] <= 3]
  mylist = []
  for idx in _df['index'].unique():
    top = _df.loc[_df['index'] == idx].head(3)
    tail = _df.loc[_df['index'] == idx].tail(3)
    top_and_tail = pd.concat([top, tail], axis=0)
    mylist.append(top_and_tail)
  comp_var.append(mylist)

In [13]:
# Examine PCA components to see what each component is comprised of - eg actual_duration
comp_var[0][0]
# First component appears to have a positive association with earlier flights where crew group did not change.

,index,Explained_Variance,variable,value
5624,1,0.161588,Crew_Group_imputed_no change_6,0.114907
483664,1,0.161588,minmax_transf_departure_hour_6,0.111315
484367,1,0.161588,minmax_transf_departure_hour_7,0.110347
487179,1,0.161588,route_quant_transf_actual_duration_1,-0.318596
486476,1,0.161588,route_quant_transf_actual_duration_0,-0.331122
488585,1,0.161588,route_quant_transf_actual_duration_2,-0.351610


In [14]:
comp_var[0][1] # Second component for actual_duration
# Second component appears to have a positive association with flights departing from East Carmen as the first or third flight, with negative association where East Carmen is the 2nd, 4th or 6th flight.
# (Zero indexing means 0 maps to 1, 2 to 3 and so forth.)

,index,Explained_Variance,variable,value
21794,2,0.088353,airport_0_EAC,0.306357
113887,2,0.088353,airport_2_EAC,0.305280
217228,2,0.088353,airport_4_EAC,0.292244
268547,2,0.088353,airport_5_EAC,-0.287401
58350,2,0.088353,airport_1_EAC,-0.333437
167315,2,0.088353,airport_3_EAC,-0.333881


In [15]:
comp_var[0][2] # Third component for actual_duration
# Third component appears to have a positive association with the last flight of the day.

,index,Explained_Variance,variable,value
488587,3,0.05589,route_quant_transf_actual_duration_2,0.238795
487181,3,0.05589,route_quant_transf_actual_duration_1,0.134764
471715,3,0.05589,last_flight_indicator_3,0.131606
426723,3,0.05589,duration_bin_1_5,-0.243443
426020,3,0.05589,duration_bin_1_4,-0.248739
424614,3,0.05589,duration_bin_1_2,-0.269269


In [16]:
# Components for departure delay: 1
comp_var[1][0]
# First component appears to have a positive association with flights departing from East Carmen as the first or third flight, with negative association where East Carmen is the 2nd, 4th or 6th flight.

,index,Explained_Variance,variable,value
21793,1,0.112734,airport_0_EAC,0.309896
113886,1,0.112734,airport_2_EAC,0.308312
217227,1,0.112734,airport_4_EAC,0.293871
268546,1,0.112734,airport_5_EAC,-0.299146
58349,1,0.112734,airport_1_EAC,-0.334000
167314,1,0.112734,airport_3_EAC,-0.334941


In [17]:
# Components for departure delay: 2
comp_var[1][1]
# Second component appears to have a positive association with flights where the crew group did not change, these were earlier flights

,index,Explained_Variance,variable,value
5625,2,0.070663,Crew_Group_imputed_no change_6,0.243566
483665,2,0.070663,minmax_transf_departure_hour_6,0.242930
404226,2,0.070663,change_reason_code_imputed_other problem_6,0.215311
9140,2,0.070663,ac_registration_quant_transf_dep_delay_1,-0.104167
8437,2,0.070663,ac_registration_quant_transf_dep_delay_0,-0.105027
472417,2,0.070663,last_flight_indicator_4,-0.130487


In [18]:
# Components for actual ground time: 1
comp_var[2][0]
# First component appears to have a positive association with flights departing from East Carmen as the first or third flight, with negative association where East Carmen is the 2nd, 4th or 6th flight.

,index,Explained_Variance,variable,value
21793,1,0.120717,airport_0_EAC,0.312405
113886,1,0.120717,airport_2_EAC,0.310842
217227,1,0.120717,airport_4_EAC,0.296425
268546,1,0.120717,airport_5_EAC,-0.298095
58349,1,0.120717,airport_1_EAC,-0.333278
167314,1,0.120717,airport_3_EAC,-0.334009


In [19]:
# Components for actual ground time: 2
comp_var[2][1]
# Second component appears to have a positive association with flights where the crew group did not change, these were earlier flights

,index,Explained_Variance,variable,value
483665,2,0.073081,minmax_transf_departure_hour_6,0.266770
5625,2,0.073081,Crew_Group_imputed_no change_6,0.254299
426722,2,0.073081,duration_bin_1_5,0.244060
430940,2,0.073081,duration_bin_2_1,-0.098687
471714,2,0.073081,last_flight_indicator_3,-0.114784
472417,2,0.073081,last_flight_indicator_4,-0.135433


In [20]:
# Create principal components to reduce dimensionality - will export to kfold build.
pca_list = []
for _df, pca_num in zip(X_for_pca, pca_nums.values()):
  X = _df.copy(deep=True)
  X.drop(columns=['day_of_origin', 'ac_registration'], inplace=True)
  pca = PCA(pca_num)
  pca.fit(X)
  X_pca = pca.transform(X)
  assert X_pca.shape[1] == pca_num, 'Error'
  pca_df = pd.DataFrame(X_pca)

  pca_list.append(pca_df)

In [21]:
# Save to pickle file for further use
filename = 'X_test_for_kfold.pkl'
with open(filename, 'wb') as file:
  pickle.dump(pca_list, file)